#复制、解压缩数据和标签

In [0]:
#从My Drive 中复制文件
import shutil

In [2]:
#本地运行时忽略
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#本地运行时忽略 否则去`drive/My Drive/Colab Notebooks/Data/ISIC2018_Task3_Training_*.zip` to /path/to/data
shutil.copy2("drive/My Drive/Colab Notebooks/Data/ISIC2018_Task3_Training_GroundTruth.zip", "./")
shutil.copy2("drive/My Drive/Colab Notebooks/Data/ISIC2018_Task3_Training_Input.zip", "./")

'./ISIC2018_Task3_Training_Input.zip'

In [0]:
!unzip -q ISIC2018_Task3_Training_GroundTruth.zip

In [0]:
!unzip -q ISIC2018_Task3_Training_Input.zip

#归因

In [6]:
print(open("ISIC2018_Task3_Training_GroundTruth/ATTRIBUTION.txt").read())

ISIC-2018 Challenge: Task 3: Training Ground Truth (HAM10000 Dataset: https://arxiv.org/abs/1803.10417 ) (c) by ViDIR Group, Department of Dermatology, Medical University of Vienna

ISIC-2018 Challenge: Task 3: Training Ground Truth (HAM10000 Dataset: https://arxiv.org/abs/1803.10417 ) is licensed under a
Creative Commons Attribution-NonCommercial 4.0 International License.

You should have received a copy of the license along with this
work. If not, see <http://creativecommons.org/licenses/by-nc/4.0/>.



#使标签/数据FAST.AI准备就绪

In [0]:
import os
import math
import pandas as pd
import numpy as np

In [0]:
np.random.seed(42)

In [0]:
df=pd.read_csv("ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv")

In [11]:
#统计六种黑色素瘤疾病所占比例
for col in df.columns:
  if col !="image":
    s=df[col].sum()
    c=df[col].count()
    print("Column:{}\t Count:{}\t Percentage:{}\t".format(col,s,round((s/c)*100,2)))
 

Column:MEL	 Count:1113.0	 Percentage:11.11	
Column:NV	 Count:6705.0	 Percentage:66.95	
Column:BCC	 Count:514.0	 Percentage:5.13	
Column:AKIEC	 Count:327.0	 Percentage:3.27	
Column:BKL	 Count:1099.0	 Percentage:10.97	
Column:DF	 Count:115.0	 Percentage:1.15	
Column:VASC	 Count:142.0	 Percentage:1.42	


In [0]:
#图像类别位于各类 并创建一个新的列名叫class 包含数据0-6
df["class"]=df.drop("image",axis=1).apply(lambda x:x.to_numpy().argmax(),axis=1)

In [14]:
#每个黑色素瘤在 class 中对应的值
a=list(df.columns)
for i in ["image","class"]:
  a.remove(i)
a=dict(zip(a,range(len(a))))
print(a)

{'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}


In [0]:
df.drop(a.keys(),axis=1,inplace=True)

In [0]:
df.to_csv("isic_labels.csv",index=False)

#提取验证数据集

In [0]:
test_idx=[]
for c in df["class"].unique():
  #获取特定类的所有图像名称
  temp=df[df['class'].apply(lambda x:True if x == c else False)]
  #从每个类别提取10%的图片
  test_idx+=temp.sample(math.floor(temp.shape[0]/10)).index.to_list()
  

In [0]:
#创建一个数据测试帧
test_df=df.iloc[test_idx].reset_index(drop=True)

In [0]:
#创建一个数据训练帧
train_df=df.drop(labels=test_idx).reset_index(drop=True)

In [0]:
train_df.to_csv("isic_train_labels.csv",index=False)
test_df.to_csv("isic_test_labels.csv",index=False)